In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import altair as alt
alt.data_transformers.enable('data_server')
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline

import math

## Vintage Analysis on User Credit Data

### Import Data

In [2]:
credit_df = pd.read_csv("credit_record.csv")

### Process Credit Data

First make the following assumption that an account is considered "bad" if it exceeds 60 days past due  
Now we can process the data so that we can graph the "bad rate" of accounts in each opening window  

In [9]:
creditdfc = credit_df.copy()

# Map bad values to the STATUS column
statmap = {
    "0":0,
    "1":0,
    "2":1,
    "3":1,
    "4":1,
    "5":1,
    "C":0,
    "X":0
}

creditdfc["STATUS"] = creditdfc["STATUS"].map(statmap)

# Create a column containing opening month of each account
mapdict = {}
for id, df in creditdfc.groupby("ID"):
    start_month = df["MONTHS_BALANCE"].min()
    mapdict[id] = start_month

# Map "bad" state to the STATUS column
creditdfc["acct_open_month"] = creditdfc["ID"].map(mapdict)

# Calculate the months since the account was opened
creditdfc["months_since_open"] = creditdfc["MONTHS_BALANCE"] - creditdfc["acct_open_month"]

# Now split the opening month into cohort bins of 1 year each 
creditdfc["acct_open_bins"] = pd.cut(creditdfc["acct_open_month"], 5)

In [11]:
# Group the data frame by the months since open and account open month
# calcualte the bad rate for each row

creditgrouped = creditdfc.groupby(["months_since_open", "acct_open_bins"])["STATUS"].agg(['sum','count'])
creditgrouped = creditgrouped[creditgrouped["count"] > 0].reset_index()
creditgrouped["pct_bad"] = creditgrouped["sum"]/creditgrouped["count"]
creditgrouped.sort_values(by = "months_since_open", inplace = True)
creditgrouped["acct_open_bins"] = creditgrouped[ "acct_open_bins"].astype(str)
creditgrouped

,months_since_open,acct_open_bins,sum,count,pct_bad
0,0,"(-60.06, -48.0]",0,6390,0.000000
1,0,"(-48.0, -36.0]",1,8157,0.000123
2,0,"(-36.0, -24.0]",0,9459,0.000000
3,0,"(-24.0, -12.0]",0,11139,0.000000
4,0,"(-12.0, 0.0]",0,10840,0.000000
...,...,...,...,...,...
176,56,"(-60.06, -48.0]",8,1217,0.006574
177,57,"(-60.06, -48.0]",7,936,0.007479
178,58,"(-60.06, -48.0]",6,698,0.008596
179,59,"(-60.06, -48.0]",4,456,0.008772


In [12]:
creditgroupedplot = creditgrouped[creditgrouped["months_since_open"] <= 40]

base = alt.Chart(creditgroupedplot).mark_line().encode(
            x=alt.X('months_since_open', title = "Months since Acct Open"),
            y=alt.Y("pct_bad", title = "bad rate"),
            color = alt.Color("acct_open_bins")
        )
base

alt.Chart(...)

The majority of bad accounts materialize after 24 months for our dataset, so we will consider a performance window of 18 months after an account opens.

Now we define our final "bad" definition as an account that exceeds 60 days past due in the first 18 months of account opening